In [1]:
#
# Big Data Exam - Spark (Parts 1 & 2 - RDDs)
# June 12, 2025
#

In [2]:
buildings_path="data/Buildings.csv"
historical_path="data/HistoricalEnergyConsumption.csv"
forecast_path="data/EnergyConsumptionForecast.csv"

In [3]:
#########
# Part 1
#########

In [4]:
buildingsRDD = sc.textFile(buildings_path)
historicalRDD = sc.textFile(historical_path)
forecastRDD = sc.textFile(forecast_path)

In [5]:
# Select the records with kwh>0
filteredHistoricalRDD = historicalRDD.filter(lambda s: float(s.split(",")[2])>0)

In [6]:
def BIDTimestampKWH(line):
    fields = line.split(",")
    BID=fields[0]
    timestamp =fields[1]
    kwh=fields[2]

    return ( (BID, timestamp), float(kwh))

In [7]:
# Join historical and forecast data on (BuildingID, Timestamp)
# RDD: ((BuildingID, Timestamp), (Historical_kWh, Forecast_kWh))
HistoricalForecast_rdd = historicalRDD.map(BIDTimestampKWH)\
                                .join(forecastRDD.map(BIDTimestampKWH))

In [8]:
# Select the pairs with an absolute percentage difference higher than 20%. 
selectedBidTimestamps = HistoricalForecast_rdd.filter(lambda p: abs(p[1][1]-p[1][0])/p[1][0]>0.2)

In [9]:
# Join selectedBidTimestamps with buildings on BuildingID to retrieve 
# energy efficiency class and type of building
#
# map buildingsRDD to (BID, (energy efficiency class, type of building))
#
# map selectedBidTimestamps to (BID, +1)
#
def BID_EEC_BT(line):
    fields = line.split(",")
    BID=fields[0]
    EEC =fields[6] # energy efficiency class
    BT=fields[4] # type of building

    return (BID, (EEC, BT))


selectedBidTimestampsWithEEC_BT = buildingsRDD.map(BID_EEC_BT)\
                                    .join(selectedBidTimestamps.map(lambda p: (p[0][0], +1)))

In [10]:
# Count the number of occurrences for each combination (energy efficiency class, type of building)
res1=selectedBidTimestampsWithEEC_BT.values()\
                                    .reduceByKey(lambda v1,v2: v1+v2)

In [11]:
res1.saveAsTextFile("output1/")

In [12]:
#########
# Part 2
#########

In [13]:
# Consumption per Sqm calculation
# Join on BID and calculate consumption per square meter for each record.
#
# Finally, map to pairs (Year, (BuildingID, ConsumptionPerSqm))

def BID_Sqm(line):
    fields = line.split(",")
    BID=fields[0]
    sqm=float(fields[3])

    return (BID, sqm)


def BID_YearKwh(line):
    fields = line.split(",")
    BID=fields[0]
    year=fields[1].split("/")[0]
    kwh=float(fields[2])

    return (BID, (year,kwh))


YearBIDConsumptionPerSqmRDD = buildingsRDD.map(BID_Sqm).join(historicalRDD.map(BID_YearKwh))\
                                    .map(lambda p: (p[1][1][0], (p[0],p[1][1][1]/p[1][0])))\
                                    .cache()

25/06/17 16:59:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [14]:
# For each year, find the maximum consumption per square meter.
# RDD: (Year, MaxConsumptionPerSqm)
yearlyMaxConsumptionRDD = YearBIDConsumptionPerSqmRDD \
                            .map(lambda x: (x[0], x[1][1])) \
                            .reduceByKey(max)

In [15]:
# Map YearBIDConsumptionPerSqmRDD to ((Year,ConsumptionPerSqm), BuildingID)
#
# Map yearlyMaxConsumptionRDD to ((Year, MaxConsumptionPerSqm), None)
#
# Join to select for each year the peak records.
# RDD: ((Year, ConsumptionPerSqm), (BuildingID, None))

peakRecordsRDD = YearBIDConsumptionPerSqmRDD.map(lambda p: ((p[0], p[1][1]), p[1][0]))\
                .join(yearlyMaxConsumptionRDD.map(lambda p: (p, None)))

In [17]:
# Count the number of peak records for each combination (year, building).
# Map to (Year, BuildingID, PeakConsumption), 1)
# Apply reduceByKey
# Result: ((Year, BuildingID, PeakConsumption), NumTimesAtPeak)
res2 = peakRecordsRDD \
        .map(lambda p: ((p[0][0], p[1][0], p[0][1]), 1)) \
        .reduceByKey(lambda v1, v2: v1 + v2)

In [19]:
res2.saveAsTextFile("output2/")